# API Asos

In [1]:
import pandas as pd
import json
import os
import sys
import dotenv

dotenv.load_dotenv()

sys.path.append(os.path.abspath('../'))
from src import funciones_api as fa
from src import dic

En primer lugar hacemos la llamada a la api para cada una de las marcas obtener todos sus productos:

In [2]:
lista_id = [8264, 4715, 28466, 20758, 18936, 28624]
lista_marcas = ["Mango", "VeroModa", "NAKD", "LoveTriangle", "JadedRose", "Ghospell"]

In [ ]:
fa.busqueda_vestido_marca(lista_id, lista_marcas)

Creamos un diccionario para cada marca donde almacenamos como valor una lista de dos elementos con la lista de ids y el diccionario de precios.

In [3]:
dic_info={}

for marca in lista_marcas:    
    lista_id_vestido = []
    dic_precio = {}

    with open(f"../datos/api_asos/{marca}/vestidos_{marca}.json", 'r') as file:
        response = json.load(file)

    for i in range(len(response["data"]["products"])):
        if "dress" in response["data"]["products"][i]["name"]:
            id = response["data"]["products"][i]["id"]
            lista_id_vestido.append(id)
            dic_precio[id]=response["data"]["products"][i]["price"]["current"]["value"]

    dic_info[marca]=[lista_id_vestido, dic_precio]

Para cada vestido, gacemos una llamada personalizada a la API para obtener más información.

In [9]:
fa.detalles_vestido(lista_marcas, dic_info)

Creamos un diccionario con todo los productos que hemos recolectado dentro de ASOS de las distintas marcas y creamos y almacenamos un dataframe con todos ellos.

In [14]:
dic_vestidos = fa.creacion_diccionario_asos(dic_info, lista_marcas)
df = pd.DataFrame(dic_vestidos)

Mango
VeroModa
NAKD
LoveTriangle
JadedRose
Ghospell


In [15]:
# añadimos la columna de categoria del vestido
df["nombre"] = df["nombre"].str.lower()
df["color"] = df["color"].str.lower()
df["categoria"] = df["nombre"].str.extract('(largo|midi|corto|maxi|mini)', expand=True)

# Como todos los valores de disponible son True, eliminamos la columna
df.drop(columns="disponible", inplace=True)

# Estandarizamos las tallas
df['talla'] = df['talla'].replace(dic.mapo_talla)

In [16]:
df

,nombre,marca,precio,color,talla,categoria
0,vestido midi plateado con cuello halter de len...,Mango,133.00,silver,S,midi
1,vestido midi plateado con cuello halter de len...,Mango,133.00,silver,M,midi
2,vestido midi plateado con cuello halter de len...,Mango,133.00,silver,L,midi
3,vestido midi negro con estampado floral y aber...,Mango,129.99,black,XS,midi
4,vestido midi negro con estampado floral y aber...,Mango,129.99,black,S,midi
...,...,...,...,...,...,...
896,vestido corto negro con flecos de ghospell,Ghospell,127.00,black,XS,corto
897,vestido corto negro con flecos de ghospell,Ghospell,127.00,black,S,corto
898,vestido corto negro con flecos de ghospell,Ghospell,127.00,black,M,corto
899,vestido corto negro con flecos de ghospell,Ghospell,127.00,black,M,corto


In [17]:
df.to_csv(f"../datos/api_asos/df_asos.csv")

In [18]:
df.sample(5)

,nombre,marca,precio,color,talla,categoria
581,vestido semilargo marrón con estampado de luna...,LoveTriangle,104.0,brown multi,XXS,largo
566,vestido lencero semilargo negro de corte al bi...,LoveTriangle,58.5,black,S,largo
561,vestido corto rojo guinda con detalle floral y...,LoveTriangle,67.5,cherry red,M,corto
320,vestido largo color piedra de manga larga con ...,VeroModa,69.0,stone,XL,largo
327,vestido corto negro plisado con cuello alto de...,VeroModa,63.0,black,XL,corto
